# 1. References

Title: Multi-Label Text Classification Using Scikit-multilearn: a Case Study with StackOverflow Questions

Link: https://medium.com/towards-artificial-intelligence/multi-label-text-classification-using-scikit-multilearn-case-study-with-stackoverflow-questions-768cb487ad12

In [18]:
import pandas as pd

In [31]:
#DATA_DIR = "../../data/raw/"
DATA_DIR = "../../data/processed/"
#INPUT_FILE_NAME = 'cleaned.parquet'
INPUT_FILE_NAME = 'cleaned_squashed1.parquet'


In [32]:
df = pd.read_parquet(DATA_DIR + INPUT_FILE_NAME)
# df = df[:200]  # same as df.head(10)
df.head()

,speaker,headline,description,duration,tags,transcript,WC,clean_transcript,clean_transcript_string,squash_tags
0,Al Gore,Averting the climate crisis,With the same humor and humanity he exuded in ...,0:16:17,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2281.0,"[thank, chris, truly, great, honor, opportunit...",thank chris truly great honor opportunity come...,"culture,politics,science,climate change,enviro..."
1,Amy Smith,Simple designs to save a life,Fumes from indoor cooking fires kill more than...,0:15:06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2687.0,"[term, invention, like, tell, tale, favorite, ...",term invention like tell tale favorite project...,"invention,engineering,design,global issues"
2,Ashraf Ghani,How to rebuild a broken state,Ashraf Ghani's passionate and powerful 10-minu...,0:18:45,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2506.0,"[public, dewey, long, ago, observe, constitute...",public dewey long ago observe constitute discu...,"poverty,economics,culture,politics,policy,glob..."
3,Burt Rutan,The real future of space exploration,"In this passionate talk, legendary spacecraft ...",0:19:37,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",3092.0,"[want, start, say, houston, problem, enter, se...",want start say houston problem enter second ge...,"invention,engineering,entrepreneur,design,busi..."
4,Chris Bangle,Great cars are great art,American designer Chris Bangle explains his ph...,0:20:04,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",3781.0,"[want, talk, background, idea, car, art, actua...",want talk background idea car art actually mea...,"invention,design,technology,business,art"


In [37]:
# df = df[:150]
# print(df)
# Temp delete
df.iloc[:,:10].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2382 entries, 0 to 2381
Data columns (total 10 columns):
speaker                    2378 non-null object
headline                   2378 non-null object
description                2378 non-null object
duration                   2378 non-null object
tags                       2378 non-null object
transcript                 2378 non-null object
WC                         2378 non-null float64
clean_transcript           2378 non-null object
clean_transcript_string    2378 non-null object
squash_tags                2378 non-null object
dtypes: float64(1), object(9)
memory usage: 204.7+ KB


In [38]:
df = df.dropna(subset=['squash_tags'])
df = df.reset_index(drop=True)
df.iloc[:,:10].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2378 entries, 0 to 2377
Data columns (total 10 columns):
speaker                    2378 non-null object
headline                   2378 non-null object
description                2378 non-null object
duration                   2378 non-null object
tags                       2378 non-null object
transcript                 2378 non-null object
WC                         2378 non-null float64
clean_transcript           2378 non-null object
clean_transcript_string    2378 non-null object
squash_tags                2378 non-null object
dtypes: float64(1), object(9)
memory usage: 185.9+ KB


In [65]:
def print_full_dataframe(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
    
def compute_tag_ratio(target_column, df=df):
    tags = df[target_column].str.replace(', ',',').str.lower().str.strip()
    split_tags = tags.str.split(',')
    tag_counts_per_talk = split_tags.apply(len)

    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags = pd.Series(joined_tags)

    tag_counts = all_tags.value_counts().rename_axis(target_column).reset_index(name='counts')
    tag_counts['no_count'] = len(df)-tag_counts['counts']
    tag_counts['ratio'] = tag_counts['counts']/tag_counts['no_count']
    tag_counts['overall_ratio'] = tag_counts['counts']/(tag_counts['no_count'] + tag_counts['counts'])
    return tag_counts

print(compute_tag_ratio('squash_tags', df))
squashed_tag_counts = compute_tag_ratio('squash_tags', df)
print_full_dataframe(squashed_tag_counts)

      squash_tags  counts  no_count     ratio  overall_ratio
0      technology     693      1685  0.411276       0.291421
1         science     522      1856  0.281250       0.219512
2   global issues     483      1895  0.254881       0.203112
3         culture     470      1908  0.246331       0.197645
4          design     399      1979  0.201617       0.167788
..            ...     ...       ...       ...            ...
95        illness      50      2328  0.021478       0.021026
96      parenting      49      2329  0.021039       0.020606
97         policy      49      2329  0.021039       0.020606
98         family      48      2330  0.020601       0.020185
99      adventure      47      2331  0.020163       0.019765

[100 rows x 5 columns]
           squash_tags  counts  no_count     ratio  overall_ratio
0           technology     693      1685  0.411276       0.291421
1              science     522      1856  0.281250       0.219512
2        global issues     483      1895  0.25

In [39]:
df_x = df[['headline', 'clean_transcript_string']]
df_y = df[['squash_tags']]

In [40]:
print(df_y.isnull().values.any())
print(df_y[df_y['squash_tags'].isnull()])
print(df.iloc[1307])

False
Empty DataFrame
Columns: [squash_tags]
Index: []
speaker                                                      Janine Shepherd
headline                                 A broken body isn't a broken person
description                Cross-country skier Janine Shepherd hoped for ...
duration                                                             0:18:57
tags                       body language,health,health care,TEDx,storytel...
transcript                 0:11\r\r\rLife is about opportunities \rcreati...
WC                                                                      3091
clean_transcript           [life, opportunity, create, embrace, olympic, ...
clean_transcript_string    life opportunity create embrace olympic dream ...
squash_tags                         health,health care,storytelling,identity
Name: 1307, dtype: object


In [42]:
from sklearn.preprocessing import MultiLabelBinarizer

y = []
for index, row in df_y.iterrows():
    y.append(set(row['squash_tags'].split(',')))
    
mlb = MultiLabelBinarizer()
encoded_y = mlb.fit_transform(y)

In [45]:
print(encoded_y[0])
print(len(encoded_y[0]))
print(mlb.inverse_transform(encoded_y)[:10])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0]
100
[('climate change', 'culture', 'environment', 'global issues', 'politics', 'science', 'sustainability', 'technology'), ('design', 'engineering', 'global issues', 'invention'), ('business', 'culture', 'economics', 'entrepreneur', 'global development', 'global issues', 'policy', 'politics', 'poverty'), ('business', 'design', 'engineering', 'entrepreneur', 'invention'), ('art', 'business', 'design', 'invention', 'technology'), ('biodiversity', 'biology', 'biotech', 'ecology', 'entrepreneur', 'genetics', 'invention', 'oceans', 'science', 'technology'), ('computers', 'entertainment', 'media', 'music', 'performance', 'technology'), ('architecture', 'cities', 'collaboration', 'culture', 'design'), ('business', 'education', 'innovation', 'invention', 'robots', 'science', 'social change', '

In [46]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
from sklearn import utils as skl_utils
from tqdm import tqdm

import multiprocessing
import numpy as np

from gensim import utils
import gensim.parsing.preprocessing as gsp

filters = [
           gsp.strip_tags, 
           gsp.strip_punctuation,
           gsp.strip_multiple_whitespaces,
           gsp.strip_numeric,
           gsp.remove_stopwords, 
           gsp.strip_short, 
           gsp.stem_text
          ]

def clean_text(s):
    s = s.lower()
    s = utils.to_unicode(s)
    for f in filters:
        s = f(s)
    return s

class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, learning_rate=0.02, epochs=20, field=None):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self._model = None
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.field = field

    def fit(self, df_x, df_y=None):
        tagged_x = [TaggedDocument(clean_text(row[str(self.field)]).split(), [index]) for index, row in df_x.iterrows()]
        model = Doc2Vec(documents=tagged_x, vector_size=self.vector_size, workers=self.workers)

        for epoch in range(self.epochs):
            model.train(skl_utils.shuffle([x for x in tqdm(tagged_x)]), total_examples=len(tagged_x), epochs=1)
            model.alpha -= self.learning_rate
            model.min_alpha = model.alpha

        self._model = model
        return self

    def transform(self, df_x):
        return np.asmatrix(np.array([self._model.infer_vector(clean_text(row[str(self.field)]).split())
                                     for index, row in df_x.iterrows()]))

In [23]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(df_x, encoded_y)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score


In [25]:
fu = FeatureUnion(transformer_list=[('title_doc2vec',Doc2VecTransformer(field='headline')),
                                    ('body_doc2vec',Doc2VecTransformer(field='clean_transcript_string'))])
binary_rel_model = BinaryRelevance(RandomForestClassifier(n_jobs=-1, n_estimators=10))

multi_label_rf_br_model = Pipeline(steps=[
                           ('feature_union', fu),
                           ('binary_relevance', binary_rel_model)
                        ])

In [26]:
import sklearn.metrics as metrics

def hamming_loss(multi_label_model_pipeline,train_x, train_y, test_x, test_y):
    predictions_test_y = multi_label_model_pipeline.predict(test_x)
    return metrics.hamming_loss(y_true=test_y, y_pred=predictions_test_y)

In [27]:
multi_label_rf_br_model.fit(train_x, train_y)
print('Hamming loss for test data :', hamming_loss(multi_label_rf_br_model,train_x,train_y,test_x,test_y))

100%|██████████| 1789/1789 [00:00<00:00, 1914186.19it/s]


Hamming loss for test data : 0.03740303004781823


In [28]:
predictions_test_y = multi_label_rf_br_model.predict(test_x)
print(predictions_test_y.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [29]:
# print(predictions_test_y.toarray()[-3])
# print(encoded_y[154])
# print(mlb.inverse_transform(predictions_test_y.toarray()))
print(test_y[27])
'''
103  want year opportunity close conference incredi...  
31   think podium bite scar chris ask tell structur...  
41   music music end end hi sirena year old connect...  
93   thank get story arrive plane long journey west...  
152  go try view world problem opportunity face ask...  
2    public dewey long ago observe constitute discu...  
154                                              music  
124  consider storyteller tell story usual way sens...  
94 
'''

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


'\n103  want year opportunity close conference incredi...  \n31   think podium bite scar chris ask tell structur...  \n41   music music end end hi sirena year old connect...  \n93   thank get story arrive plane long journey west...  \n152  go try view world problem opportunity face ask...  \n2    public dewey long ago observe constitute discu...  \n154                                              music  \n124  consider storyteller tell story usual way sens...  \n94 \n'

In [30]:
index = -1
print(test_y[index])
print(mlb.inverse_transform(test_y)[index])
mlb.inverse_transform(predictions_test_y)[index]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
('design', 'green', 'sustainability')


('technology',)

In [47]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.multiclass import OneVsRestClassifier
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import f1_score

In [48]:
X = df['clean_transcript_string']

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y)

In [50]:
from sklearn.base import BaseEstimator, TransformerMixin

# Trying to fix onevsrest param forwarding

class OVR(BaseEstimator, TransformerMixin):
    def __init__(self, clf=None):
        self.clf = OneVsRestClassifier(clf())
    
    def fit(self, X, y):
        if self.clf is not None:
            self.clf.fit(X, y)

        return self
    
    def predict(self, X):
        return self.clf.predict(X)
    
    def predict_proba(self, X):
        return self.clf.predict_proba(X)

## On why we need to use estimator__ as name

https://stackoverflow.com/questions/12632992/gridsearch-for-an-estimator-inside-a-onevsrestclassifier/12637528#12637528

In [51]:
# Set grid search params

# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_fl = [1.0, 0.5, 0.1]

cv_grid_params = [{'vectorizer__min_df': np.linspace(0.005, 0.05, 5),
                   'vectorizer__ngram_range': ((1, 1),(1, 2))
                  }] # Not implemened yet

tfidf_grid_params = [] # Not implemented yet

lr_grid_params = [{'clf__estimator__penalty': ['l1', 'l2'],
                   'clf__estimator__C': param_range_fl,
                   'clf__estimator__solver': ['liblinear']
                  }] 

rf_grid_params = [{'clf__estimator__criterion': ['gini', 'entropy'],
                   'clf__estimator__min_samples_leaf': param_range,
                   'clf__estimator__max_depth': param_range,
                   'clf__estimator__min_samples_split': param_range[1:],
                   'clf__estimator__n_estimators': [10]
                  }]

svm_grid_params = [{'clf__estimator__kernel': ['linear', 'rbf'], 
                    'clf__estimator__C': param_range, # np.logspace(-1, 2, 10),
                    'clf__estimator__gamma': ['auto'], # np.logspace(-1, 1, 10)
                    'clf__estimator__probability': [True]
                   }]

mnb_grid_params = []

scoring = 'accuracy'
folds = 5
njobs = -1


# Instantiate vectorizer and desired models

# OVR
    ## logistic regression
cv_lr_ovr = Pipeline([('vectorizer', CountVectorizer()),
                      ('clf', OneVsRestClassifier(LogisticRegression()))
                     ]
                    )
gs_cv_lr_ovr = GridSearchCV(estimator=cv_lr_ovr,
                            param_grid=lr_grid_params,
                            scoring=scoring,
                            cv=folds) 


tfidf_lr_ovr = Pipeline([('vectorizer', TfidfVectorizer()),
                        ('clf', OneVsRestClassifier(LogisticRegression()))
                       ]
                      )
gs_tfidf_lr_ovr = GridSearchCV(estimator=tfidf_lr_ovr,
                               param_grid=lr_grid_params,
                               scoring=scoring,
                               cv=folds) 


    ## random forest
# cv_rf_ovr = Pipeline([('vectorizer', CountVectorizer()),
#                       ('clf', OVR(RandomForestClassifier()))
#                      ]
#                     )
cv_rf_ovr = Pipeline([('vectorizer', CountVectorizer()),
                      ('clf', OneVsRestClassifier(RandomForestClassifier()))
                     ]
                    )
gs_cv_rf_ovr = GridSearchCV(estimator=cv_rf_ovr,
                            param_grid=rf_grid_params,
                            scoring=scoring,
                            cv=folds, 
                            n_jobs=njobs)


# tfidf_rf_ovr = Pipeline([('vectorizer', TfidfVectorizer()),
#                         ('clf', OVR(RandomForestClassifier()))
#                        ]
#                       )
tfidf_rf_ovr = Pipeline([('vectorizer', TfidfVectorizer()),
                        ('clf', OneVsRestClassifier(RandomForestClassifier()))
                       ]
                      )
gs_tfidf_rf_ovr = GridSearchCV(estimator=tfidf_rf_ovr,
                               param_grid=rf_grid_params,
                               scoring=scoring,
                               cv=folds, 
                               n_jobs=njobs)


    ## support vector classifier
cv_svm_ovr = Pipeline([('vectorizer', CountVectorizer()),
                       ('clf', OneVsRestClassifier(SVC()))
                      ]
                     )
gs_cv_svm_ovr = GridSearchCV(estimator=cv_svm_ovr,
                             param_grid=svm_grid_params,
                             scoring=scoring,
                             cv=folds,
                             n_jobs=njobs)


tfidf_svm_ovr = Pipeline([('vectorizer', TfidfVectorizer()),
                          ('clf', OneVsRestClassifier(SVC()))
                         ]
                        )
gs_tfidf_svm_ovr = GridSearchCV(estimator=tfidf_svm_ovr,
                                param_grid=svm_grid_params,
                                scoring=scoring,
                                cv=folds,
                                n_jobs=njobs)


    ## multinomial naive bayes
cv_mnb_ovr = Pipeline([('vectorizer', CountVectorizer()),
                       ('clf', OneVsRestClassifier(MultinomialNB()))
                      ]
                     )
gs_cv_svm_ovr = GridSearchCV(estimator=cv_mnb_ovr,
                             param_grid=mnb_grid_params,
                             scoring=scoring,
                             cv=folds,
                             n_jobs=njobs)


tfidf_mnb_ovr = Pipeline([('vectorizer', TfidfVectorizer()),
                          ('clf', OneVsRestClassifier(MultinomialNB()))
                         ]
                        )
gs_tfidf_mnb_ovr = GridSearchCV(estimator=tfidf_mnb_ovr,
                                param_grid=mnb_grid_params,
                                scoring=scoring,
                                cv=folds,
                                n_jobs=njobs)

## This style might be more efficient, 
http://scikit.ml/api/skmultilearn.problem_transform.br.html

https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

http://scikit.ml/stratification.html

In [ ]:
# TODO: 
# 1. Check if TfidfTransformer use_idf=False is the same as Countvectorizer? or there are other metrics to suppress
# 2. Get scoring function to work, hamming? -- kinda done
# 3. Balanced class labels

# Binaryrelevance

# param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range = [1, 2, 3, 4, 5]
param_range_lr = [1.0, 0.5, 0.1]

# Set params, comment out as see fit

vectorizer_params = {
#     'vectorizer__min_df': np.linspace(0.005, 0.05, 5),
#     'vectorizer__ngram_range': [(1, 1), (1, 2)], # This shit blows up your memory
#     'tfidf__norm': ('l1', 'l2'),
    'tfidf__use_idf': [True, False],
}

lr_params = {
    'clf__classifier': [LogisticRegression()],
        'clf__classifier__penalty': ['l1', 'l2'],
        'clf__classifier__C': param_range_lr,
        'clf__classifier__solver': ['liblinear']
}

svc_params = {
    'clf__classifier': [SVC()],
        'clf__classifier__kernel': ['linear', 'rbf'],
        'clf__classifier__C': param_range, # np.logspace(-1, 2, 10),
        'clf__classifier__gamma': ['auto'], # np.logspace(-1, 1, 10)
        'clf__classifier__probability': [True],
}

rf_params = {
    'clf__classifier': [RandomForestClassifier()],
        'clf__classifier__criterion': ['gini', 'entropy'],
        'clf__classifier__min_samples_leaf': param_range,
        'clf__classifier__max_depth': param_range,
        'clf__classifier__min_samples_split': param_range[1:],
        'clf__classifier__n_estimators': [10],
}

mnb_params = {
    'clf__classifier': [MultinomialNB()],
        'clf__classifier__alpha': [0.7, 1.0],
}

## Stack params
parameters = [
#     {**vectorizer_params, **lr_params},
#     {**vectorizer_params, **svc_params},
#     {**vectorizer_params, **rf_params},
    {**vectorizer_params, **mnb_params}
]

br_pipeline = Pipeline([('vectorizer', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf', BinaryRelevance()),
#                         ('clf', OneVsRestClassifier()),
                       ]
                      )

# Gridsearch settings
# scoring = make_scorer(f1_score, average='micro') # possible scorings 'f1_micro' 'f1_macro'
# scoring = 'f1_micro'
scoring = make_scorer(hamming_loss)
folds = 3
njobs = -1

model = GridSearchCV(br_pipeline, parameters, scoring=scoring, cv=folds, n_jobs=njobs)

In [ ]:
model.fit(X_train,y_train)
print (model.best_params_, model.best_score_)
pd.DataFrame(model.cv_results_)

In [52]:
# pipelines = [cv_lr_ovr, tfidf_lr_ovr, 
#              cv_rf_ovr, tfidf_rf_ovr, 
#              cv_svm_ovr, tfidf_svm_ovr]

# pipelines = [cv_rf_ovr, tfidf_rf_ovr]
pipelines = [cv_lr_ovr, tfidf_lr_ovr]
# grids = [gs_cv_lr_ovr, gs_tfidf_lr_ovr,
#          gs_cv_rf_ovr, gs_tfidf_rf_ovr,
#          gs_cv_svm_ovr, gs_tfidf_svm_ovr]
grids = [gs_cv_lr_ovr, gs_tfidf_lr_ovr]

## Beware the next line

### 

In [53]:
%%time
# for pipe in pipelines:
#     pipe.fit(X_train, y_train)

for model in grids:
    model.fit(X_train,y_train)

Wall time: 14min 47s


In [54]:
for idx, clf in enumerate(grids):
    y_pred_prob = clf.predict_proba(X_test)
    t = 0.1 # threshold value
    y_pred_new = (y_pred_prob >= t).astype(int)
    scoring = f1_score(y_test, y_pred_new, average="micro")
    print('%s pipeline test accuracy: %.3f' % (idx, scoring))

0 pipeline test accuracy: 0.386
1 pipeline test accuracy: 0.389


In [64]:
mlb.inverse_transform(y_pred_new)[3]

('ai',
 'computers',
 'culture',
 'entertainment',
 'future',
 'global issues',
 'humanity',
 'innovation',
 'intelligence',
 'science',
 'technology',
 'tedx')

In [ ]:
best_acc = 0.0
best_clf = 0
best_pipe = ''
for idx, clf in enumerate(pipelines):
    if clf.score(X_test, y_test) > best_acc:
        best_acc = clf.score(X_test, y_test)
        best_pipe = clf
print(f'Classifier with best accuracy: {best_pipe.named_steps} \n with accuracy of {best_acc}')
# joblib.dump(best_pipe, 'best_classifier.pkl', compress=1)

# Evaluation

In [55]:
y_test

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [56]:
y_pred_new

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [67]:
print(mlb.inverse_transform(y_test)[2])
print(mlb.inverse_transform(y_pred_new)[2])
list(mlb.classes_) # So that we can map back the precision, recall and F1 to each tag

('education', 'family', 'global issues', 'women')
('activism', 'children', 'culture', 'education', 'entertainment', 'family', 'global issues', 'humanity', 'inequality', 'parenting', 'social change', 'women')


['activism',
 'adventure',
 'africa',
 'animals',
 'architecture',
 'art',
 'beauty',
 'big problems',
 'biodiversity',
 'biology',
 'biotech',
 'brain',
 'business',
 'children',
 'cities',
 'climate change',
 'cognitive science',
 'collaboration',
 'communication',
 'community',
 'computers',
 'creativity',
 'culture',
 'data',
 'demo',
 'design',
 'disease',
 'ecology',
 'economics',
 'education',
 'energy',
 'engineering',
 'entertainment',
 'entrepreneur',
 'environment',
 'evolution',
 'exploration',
 'family',
 'film',
 'food',
 'future',
 'genetics',
 'global development',
 'global issues',
 'government',
 'green',
 'happiness',
 'health',
 'health care',
 'history',
 'humanity',
 'humor',
 'identity',
 'illness',
 'inequality',
 'innovation',
 'internet',
 'invention',
 'language',
 'leadership',
 'life',
 'live music',
 'math',
 'media',
 'medical research',
 'medicine',
 'mental health',
 'mind',
 'motivation',
 'music',
 'nature',
 'neuroscience',
 'oceans',
 'parenting',
 

In [63]:
def compute_tp_tn_fp_fn(y_test, y_pred, classes):
    '''
    Return:
    pre_score = {
        'tag_1': {
            'index': ,
            'tp': ,
            'tn': ,
            'fp': ,
            'fn': 
        }
    }
    '''
    # Create dictionary of tags 
    pre_score = {}
    for index_tag, tag in enumerate(classes):
        pre_score[tag] = {
            'index':index_tag,
            'tp': 0,
            'tn': 0,
            'fp': 0,
            'fn': 0
        }
    for transcript_index, transcript_value in enumerate(y_test):
        for tag_index, tag_value_test in enumerate(transcript_value):
            if tag_value_test == y_pred[transcript_index][tag_index] and tag_value_test == 1:
                pre_score[classes[tag_index]]['tp'] += 1
            elif tag_value_test == y_pred[transcript_index][tag_index] and tag_value_test == 0:
                pre_score[classes[tag_index]]['tn'] += 1
            elif tag_value_test != y_pred[transcript_index][tag_index] and tag_value_test == 1:
                pre_score[classes[tag_index]]['fn'] += 1
            elif tag_value_test != y_pred[transcript_index][tag_index] and tag_value_test == 0:
                pre_score[classes[tag_index]]['fp'] += 1
    return pre_score
scores_preprocess = compute_tp_tn_fp_fn(y_test, y_pred_new, mlb.classes_)

In [64]:
 print(scores_preprocess)

{'activism': {'index': 0, 'tp': 2, 'tn': 534, 'fp': 26, 'fn': 33}, 'adventure': {'index': 1, 'tp': 0, 'tn': 587, 'fp': 1, 'fn': 7}, 'africa': {'index': 2, 'tp': 13, 'tn': 563, 'fp': 7, 'fn': 12}, 'animals': {'index': 3, 'tp': 15, 'tn': 546, 'fp': 19, 'fn': 15}, 'architecture': {'index': 4, 'tp': 7, 'tn': 566, 'fp': 12, 'fn': 10}, 'art': {'index': 5, 'tp': 37, 'tn': 499, 'fp': 46, 'fn': 13}, 'beauty': {'index': 6, 'tp': 0, 'tn': 581, 'fp': 1, 'fn': 13}, 'big problems': {'index': 7, 'tp': 2, 'tn': 582, 'fp': 2, 'fn': 9}, 'biodiversity': {'index': 8, 'tp': 5, 'tn': 569, 'fp': 8, 'fn': 13}, 'biology': {'index': 9, 'tp': 37, 'tn': 497, 'fp': 47, 'fn': 14}, 'biotech': {'index': 10, 'tp': 6, 'tn': 564, 'fp': 7, 'fn': 18}, 'brain': {'index': 11, 'tp': 29, 'tn': 533, 'fp': 19, 'fn': 14}, 'business': {'index': 12, 'tp': 54, 'tn': 356, 'fp': 173, 'fn': 12}, 'children': {'index': 13, 'tp': 14, 'tn': 534, 'fp': 31, 'fn': 16}, 'cities': {'index': 14, 'tp': 15, 'tn': 558, 'fp': 13, 'fn': 9}, 'climate

In [72]:
def compute_precision_recall_f1(preprocessed_scores):
    for key, value in preprocessed_scores.items():
        try:
            precision = value['tp']/(value['tp']+value['fp'])
        except:
            print('precision issue: {}'.format(key))
            precision = 0.0
        try:
            recall = value['tp']/(value['tp']+value['fn'])
        except:
            print('recall issue: {}'.format(key))
            recall = 0.0
        try:
            f1 = (2 * precision * recall)/(precision + recall)
        except:
            print('f1 issue: {}'.format(f1))
            f1=0.0
        preprocessed_scores[key]['precision'] = round(precision,2)
        preprocessed_scores[key]['recall'] = round(recall,2)
        preprocessed_scores[key]['f1'] = round(f1,2)
    return preprocessed_scores
final_scores = compute_precision_recall_f1(scores_preprocess)
print(final_scores)

f1 issue: 0.06349206349206349
f1 issue: 0.556390977443609
precision issue: demo
f1 issue: 0.2631578947368421
precision issue: motivation
f1 issue: 0.24242424242424243
precision issue: peace
f1 issue: 0.14285714285714288
f1 issue: 0.5333333333333333
precision issue: potential
f1 issue: 0.5045045045045045
{'activism': {'index': 0, 'tp': 2, 'tn': 534, 'fp': 26, 'fn': 33, 'precision': 0.07, 'recall': 0.06, 'f1': 0.06}, 'adventure': {'index': 1, 'tp': 0, 'tn': 587, 'fp': 1, 'fn': 7, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'africa': {'index': 2, 'tp': 13, 'tn': 563, 'fp': 7, 'fn': 12, 'precision': 0.65, 'recall': 0.52, 'f1': 0.58}, 'animals': {'index': 3, 'tp': 15, 'tn': 546, 'fp': 19, 'fn': 15, 'precision': 0.44, 'recall': 0.5, 'f1': 0.47}, 'architecture': {'index': 4, 'tp': 7, 'tn': 566, 'fp': 12, 'fn': 10, 'precision': 0.37, 'recall': 0.41, 'f1': 0.39}, 'art': {'index': 5, 'tp': 37, 'tn': 499, 'fp': 46, 'fn': 13, 'precision': 0.45, 'recall': 0.74, 'f1': 0.56}, 'beauty': {'index': 6, 

In [74]:
def format_scores_df(final_scores=final_scores, tag_classes=mlb.classes_):
    precision = []
    recall = []
    f1 = []
    for index, value in enumerate(tag_classes):
        precision.append(final_scores[value]['precision'])
        recall.append(final_scores[value]['recall'])
        f1.append(final_scores[value]['f1'])
    df_result = pd.DataFrame(list(zip(tag_classes, precision, recall, f1)), 
               columns =['class', 'precision', 'recall', 'f1']) 
    return df_result
df_results = format_scores_df()
print_full_dataframe(df_results)

                 class  precision  recall    f1
0             activism       0.07    0.06  0.06
1            adventure       0.00    0.00  0.00
2               africa       0.65    0.52  0.58
3              animals       0.44    0.50  0.47
4         architecture       0.37    0.41  0.39
5                  art       0.45    0.74  0.56
6               beauty       0.00    0.00  0.00
7         big problems       0.50    0.18  0.27
8         biodiversity       0.38    0.28  0.32
9              biology       0.44    0.73  0.55
10             biotech       0.46    0.25  0.32
11               brain       0.60    0.67  0.64
12            business       0.24    0.82  0.37
13            children       0.31    0.47  0.37
14              cities       0.54    0.62  0.58
15      climate change       0.31    0.27  0.29
16   cognitive science       0.25    0.29  0.27
17       collaboration       0.14    0.07  0.09
18       communication       0.25    0.38  0.30
19           community       0.22    0.1